In [13]:
import requests
import pandas as pd
import time
pd.set_option("display.max_columns", None)
from datetime import timezone, timedelta, date, datetime
from geopy.distance import geodesic #計算GPS兩端點距離
import folium #GPS數據視覺化
from global_land_mask import globe #確認GPS位置是否在陸地
import warnings
warnings.filterwarnings('ignore') # 關閉運行的警告
pd.set_option("mode.chained_assignment", None)
import pprint
import json
import tkinter as tk
from tkinter import filedialog
import gc
import sqlite3
import httpx
import numpy as np
import os
import zipfile
import pyodbc
from pd_to_mssql import to_sql
from bs4 import BeautifulSoup as bs
import lxml

In [16]:
with open(r"C:\Users\kc.hsu\Desktop\爬蟲測試.html", "w") as f:
    f.write(res.text)

In [21]:
pd.read_html(r"C:\Users\kc.hsu\Desktop\爬蟲測試.html", encoding="big5")[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          301 non-null    object
 1   汽車                  301 non-null    int64 
 2   大客車 /自用             301 non-null    int64 
 3   大客車 /營業 /遊覽車        301 non-null    object
 4   大客車 /營業 /營業(不含遊覽車)  301 non-null    int64 
 5   大貨車 /自用             301 non-null    int64 
 6   大貨車 /營業             301 non-null    int64 
 7   小客車 /自用             301 non-null    int64 
 8   小客車 /營業 /租賃         301 non-null    object
 9   小客車 /營業 /計程車        301 non-null    int64 
 10  小客車 /營業 /其他營業小客車    301 non-null    object
dtypes: int64(7), object(4)
memory usage: 26.0+ KB


## 整理四色的數據

In [ ]:
usecols = [0, 2, 5, 6, 8]
names = ["plate_numb", "brand", "power", "body", "date"]
tour_bus = pd.read_csv(r"C:\Users\kc.hsu\Downloads\tour_bus.csv", usecols=usecols, skiprows=2, names=names)
cb = pd.read_csv(r"C:\Users\kc.hsu\Downloads\bus.csv", usecols=usecols, skiprows=2, names=names)

In [ ]:
tour_bus["power_type"] = tour_bus["power"].map(lambda x: "電動" if "⚡" in x else "柴油")
cb["power_type"] = cb["power"].map(lambda x: "電動" if "⚡" in x else "柴油")

In [ ]:
result = pd.concat([cb, tour_bus]).reset_index(drop=True)

In [ ]:
result["year"] = result["date"].map(lambda x: x[:4])

In [ ]:
result = result[['plate_numb', 'brand', 'body', 'power_type', 'year']]
result.drop_duplicates(subset=["plate_numb"], inplace=True)

In [ ]:
con = sqlite3.connect("./tdx.db")
# cur = con.cursor()
# cur.execute("create table bus_info(plate_numb, brand, body, power_type, year)")
result.to_sql("bus_info", con, if_exists="append", index=False)
con.commit()
con.close()

In [2]:
## 建立每月批次整理ICB數據的class
class CB:
    use_cols = ['PlateNumb', 'OperatorID', 'PositionLon',
       'PositionLat', 'GPSTime']
    city_name = ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市", "基隆市", "新竹市", "新竹縣", "苗栗縣", "彰化縣", "南投縣", "嘉義市", "嘉義縣", 
             "雲林縣", "屏東縣", "宜蘭縣", "花蓮縣", "臺東縣", "澎湖縣", "連江縣", "金門縣"]
    city_code = ["TPE", "NWT", "TAO", "TXG", "TNN", "KHH", "KEE", "HSZ", "HSQ", "MIA", "CHA", "NAN", "CYI", "CYQ", "YUN", "PIF", "ILA", "HUA", "TTT", "PEN", 
                "LIE", "KIN"]
    city_dict = dict(zip(city_name, city_code))
    
    
    def __init__(self):
        self.run()
        # self.path = path
        # self.play = self.concur(self.path)
        # self.operator_cn_dict, self.operator_en_dict = self.update_operatorID()
        # self.brand_dict, self.body_dict = self.four_color()
    
    def update_operatorID(self):
        icb_operator_api = "https://linkmotc.map8.zone/app/data?modelName=operator_coach&skip=0&limit=&whereOptions="
        res = httpx.get(icb_operator_api)
        df = pd.read_json(res.text)
        operator_cn_dict = dict(zip(df["operator_id"], df["operator_name"]))
        operator_en_dict = dict(zip(df["operator_id"], df["operator_name_eng"]))
        return operator_cn_dict, operator_en_dict
    
    def four_color(self):
        four = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\市場資訊\大車監理數據.xlsx", sheet_name="Sheet1")
        four["廠牌"] = four["廠牌"].fillna("")
        four["車體"] = four["車體"].fillna("")
        brand_dict = dict(zip(four["車號"], four["廠牌"]))
        body_dict = dict(zip(four["車號"], four["車體"]))
        return brand_dict, body_dict
    
    def select_files(self):
        # 測試讀取檔案路徑list存進txt，看看中斷之後再次讀取是否OK
        root = tk.Tk()
        root.withdraw()
        paths = filedialog.askopenfilenames()

        # 將開啟的路徑都寫進去tdx_path.txt
        with open("cb_paths.txt", "w", encoding="utf-8") as f:
            for path in paths:
                f.write(path + "\n")
                
                
    def load_files(self):
        if os.path.isfile("./tdx.db"):
            pass
        else:
            self.create_db()
        # 打開tdx_paths.txt並讀取所有路徑
        with open("cb_paths.txt", "r", encoding="utf-8") as f:
            lines = f.readlines()

        #當路徑list的長度大於0，執行動作，並將路徑從list中刪除，一旦中斷執行，就將剩下還未執行到的路徑list重新寫回檔案中待下次繼續執行
        while len(lines) > 0:
            path = lines[0].rstrip("\n")
            
            try:
                print("Dealing with {}".format(path.split("/")[-1]))
                df = pd.read_csv(path, usecols=self.use_cols)
                print("讀取完畢============={}".format(path.split("/")[-1]))
                result = self.data_extract(df)
                print("已完成=============" + path.split("/")[-1])
                result["City"] = self.city_dict[path.split("/")[-1][:3]]
                self.write_db(result)
                print("已將============={}寫入資料庫".format(path.split("/")[-1]))
                print("移除============={}".format(lines[0].rstrip("\n")))
                lines.pop(0)                
            
            except Exception as e:
                print(e)
                lines.pop(0)
                pass
                
            except KeyboardInterrupt:
                with open("cb_paths.txt", "w", encoding="utf-8") as f:
                    for line in lines:
                        f.write(line)
                break
            
            else:
                if len(lines) == 0:
                    os.remove("cb_paths.txt")
                else:
                    pass
            
            
    def data_extract(self, df):
        data = df.sort_values(by="GPSTime", ascending=True)
        data = data[(data["PlateNumb"] != "???") & (data["PositionLon"].between(-180, 180)) & (data["PositionLat"].between(-85, 85))]
        data["NextGPS"] = data.groupby("PlateNumb")["GPSTime"].shift(-1)
        data["更新時間差"] = pd.to_datetime(data["NextGPS"]) - pd.to_datetime(data["GPSTime"])
        data["更新時間秒差"] = data["更新時間差"].map(lambda x: 0 if pd.isna(x) else x / np.timedelta64(1, "s"))
        data["Coordinates"] = data[["PositionLat", "PositionLon"]].apply(tuple, axis=1)
        data["Land"] = globe.is_land(data["PositionLat"], data["PositionLon"])
        data = data[data["Land"] == True]
        data["NextCoordinates"] = data.groupby("PlateNumb")["Coordinates"].shift(-1)
        data["Distance"] = data.apply(lambda data: geodesic(data["Coordinates"], data["NextCoordinates"]).meters if pd.notna(data["NextCoordinates"]) else 0, axis=1) / 1000
        # data = data[data["Distance"] <= 2.5]
        data["數據有效性"] = data.apply(lambda x: "有效" if  x["Distance"] <= round((130 / 3600) * x["更新時間秒差"], 4) else "無效", axis=1)
        data = data[data["數據有效性"] == "有效"]
        data["Time"] = pd.to_datetime(data["GPSTime"].map(lambda x: x.split("+")[0]), format="%Y-%m-%dT%H:%M")
        result = data.groupby(["PlateNumb", "OperatorID"]).agg({"Distance": "sum", "Time": lambda x: (x.max() - x.min()) / pd.Timedelta(hours=1)}).reset_index()
        result["Date"] = data["Time"].max().strftime("%Y-%m-%d")
        # result["AVG_Speed"] = result["Distance"] / result["Time"]
        # result["AVG_Speed"].fillna(0, inplace=True)
        result[["Distance", "Time"]] = result[["Distance", "Time"]].applymap(lambda x: round(x, 1))
        # result["業者名稱"] = result["OperatorID"].map(self.operator_dict)
        # result["業者中文名稱"] = result["OperatorID"].map(self.operator_cn_dict)
        # result["業者英文名稱"] = result["OperatorID"].map(self.operator_en_dict)
        # result["廠牌"] = result["PlateNumb"].map(self.brand_dict)
        # result["車體"] = result["PlateNumb"].map(self.body_dict)
        result = result[["PlateNumb", "OperatorID", "Distance", "Time", "Date"]]
        return result
    
    def create_db(self):
        # 第一次初始化table才需要執行
        con = sqlite3.connect("./tdx.db")
        cur = con.cursor()

        cur.execute("create table ICB(PlateNumb, OperatorID, Distance, Time, Date)")
        con.commit()
        con.close()
    
    def write_db(self, result):
        con = sqlite3.connect("./tdx.db")
        # cur = con.cursor()
        # for index, row in result.iterrows():
        #     print(tuple(row))
        #     cur.execute('INSERT INTO ICB (PlateNumb, OperatorID_CN, OperatorID_EN, Brand, Model, Distance, Time, Date, AVG_Speed) values (?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        result.to_sql('CB', con, if_exists="append", index=False)
        # del result
        # gc.collect()
        con.commit()
        con.close()
        
        
    def run(self):
        if os.path.isfile("cb_paths.txt"):
            pass
        else:
            self.select_files()
        self.load_files()
    


In [ ]:
from concurrent.futures import ThreadPoolExecutor

root = tk.Tk()
root.withdraw()
filepaths = filedialog.askopenfilenames()

with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(CB, filepaths)

In [ ]:
con = sqlite3.connect("./tdx.db")
cur = con.cursor()
cur.execute("create table CB(PlateNumb, OperatorID, Distance, Time, Date, City)")
con.commit()
con.close()

In [3]:
cb = CB()

Dealing with 新北市市區公車定時歷史資料(A1)2023-05-17.CSV
讀取完畢=============新北市市區公車定時歷史資料(A1)2023-05-17.CSV
已完成=============新北市市區公車定時歷史資料(A1)2023-05-17.CSV
已將=============新北市市區公車定時歷史資料(A1)2023-05-17.CSV寫入資料庫
移除=============D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/CB/解壓縮檔案/新北市市區公車定時歷史資料(A1)2023-05-17.CSV
Dealing with 新北市市區公車定時歷史資料(A1)2023-05-18.CSV
讀取完畢=============新北市市區公車定時歷史資料(A1)2023-05-18.CSV
已完成=============新北市市區公車定時歷史資料(A1)2023-05-18.CSV
已將=============新北市市區公車定時歷史資料(A1)2023-05-18.CSV寫入資料庫
移除=============D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/CB/解壓縮檔案/新北市市區公車定時歷史資料(A1)2023-05-18.CSV
Dealing with 新北市市區公車定時歷史資料(A1)2023-05-19.CSV
讀取完畢=============新北市市區公車定時歷史資料(A1)2023-05-19.CSV
已完成=============新北市市區公車定時歷史資料(A1)2023-05-19.CSV
已將=============新北市市區公車定時歷史資料(A1)2023-05-19.CSV寫入資料庫
移除=============D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/CB/解壓縮檔案/新北市市區公車定時歷史資料(A1)2023-05-19.CSV
Dealing with 新北市市區公車定時歷史資料(A1)2023-05-20.CSV
讀取完畢=============新北市市區公車定時歷史資料(A1)2023-05-20.CSV
已完成=============新北

In [ ]:
## 下載市區公車營運業者名單
dfs = []
city_code = ["TPE", "NWT", "TAO", "TXG", "TNN", "KHH", "KEE", "HSZ", "HSQ", "MIA", "CHA", "NAN", "CYI", "CYQ", "YUN", "ILA", "HUA", "TTT", "PEN", 
             "LIE", "KIN", "PIF"]
for city in city_code:
    path = "https://linkmotc.map8.zone/app/data?modelName=operator_bus&skip=0&limit=&whereOptions=authority_code%3D+%27" + city + "%27"
    res = requests.get(path)
    df = pd.DataFrame.from_dict(json.loads(res.text))
    dfs.append(df)

cb_operator = pd.concat(dfs, ignore_index=True)
cb_operator["operator_id"] = cb_operator["operator_id"].map(lambda x: x.replace("0", "") if x.startswith("0") else x)

In [ ]:
con = sqlite3.connect("./tdx.db")
cur = con.cursor()
cur.execute("create table cb_operator(authority_code, operator_id, operator_name, operator_name_eng)")
con.commit()
con.close()

In [ ]:
cb_sql = cb_operator[["authority_code", "operator_id", "operator_name", "operator_name_eng"]]

In [ ]:
con = sqlite3.connect("./tdx.db")
cb_sql.to_sql("cb_operator", con, if_exists="append", index=False)
con.commit()
con.close()

In [ ]:
con = sqlite3.connect("./tdx.db")
cur = con.cursor()
cur.execute("create table CB(PlateNumb, OperatorID, Distance, Time, Date, City)")
con.commit()
con.close()

## 結合數據


In [25]:
class DataConcat:
    
    today = date.today().strftime("%Y-%m-%d")  
        
    def __init__(self):
        self.cb, self.icb, self.bus_info, self.icb_operator, self.cb_operator = self.read_data()
        self.cb_operator_dict = dict(zip(list(zip(self.cb_operator["authority_code"], self.cb_operator["operator_id"])), list(zip(self.cb_operator["operator_name"], self.cb_operator["operator_name_eng"]))))
        self.car_info_dict = dict(zip(self.bus_info["plate_numb"], self.bus_info[["brand", "body", "power_type", "year"]].apply(tuple, axis=1)))
        self.icb_operator_dict = dict(zip(self.icb_operator["operator_id"].astype(str), list(zip(self.icb_operator["operator_name"], self.icb_operator["operator_name_eng"]))))
        self.city_name = ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市", "基隆市", "新竹市", "新竹縣", "苗栗縣", "彰化縣", "南投縣", "嘉義市", "嘉義縣", 
                "雲林縣", "屏東縣", "宜蘭縣", "花蓮縣", "臺東縣", "澎湖縣", "連江縣", "金門縣"]
        self.city_code = ["TPE", "NWT", "TAO", "TXG", "TNN", "KHH", "KEE", "HSZ", "HSQ", "MIA", "CHA", "NAN", "CYI", "CYQ", "YUN", "PIF", "ILA", "HUA", "TTT", "PEN", 
                    "LIE", "KIN"]
        self.city_dict = dict(zip(self.city_code, self.city_name))
        self.df = self.run()
        
        
    
    def read_data(self):
        con = sqlite3.connect("./tdx.db")
        cb = pd.read_sql("select * from CB", con)
        icb = pd.read_sql("select * from ICB", con)
        bus_info = pd.read_sql("select * from bus_info", con)
        icb_operator = pd.read_sql("select * from operator", con)
        cb_operator = pd.read_sql("select * from cb_operator", con)
        con.commit()
        con.close()
        return cb, icb, bus_info, icb_operator, cb_operator
    
    def combine_data_cb(self):
        self.cb["OperatorID"] = self.cb["OperatorID"].astype(str)
        self.cb["operator_info"] = self.cb[["City", "OperatorID"]].apply(tuple, axis=1)
        self.cb["operator_info"] = self.cb["operator_info"].map(self.cb_operator_dict)
        self.cb["info_tuple"] = self.cb["PlateNumb"].map(self.car_info_dict)
        self.cb[["operator_name", "operator_name_en"]] = pd.DataFrame(self.cb["operator_info"].tolist(), index=self.cb.index, columns=["operator_name", "operator_name_en"])
        self.cb[["Brand", "Body", "Power_type", "Year"]] = pd.DataFrame(self.cb["info_tuple"].tolist(), index=self.cb.index, columns=["Brand", "Body", "Power_type", "Year"])
        self.cb["City"] = self.cb["City"].map(self.city_dict)
        self.cb["Date"] = pd.to_datetime(self.cb["Date"], format="%Y-%m-%d")
        self.cb = self.cb.groupby([pd.Grouper(key="Date", freq="m"), "PlateNumb", "City", "operator_name", "operator_name_en", "Brand", "Body", "Power_type", "Year"])[["Distance", "Time"]].sum().reset_index()
        self.cb["Type"] = "City Bus"
        return self.cb
    
    def combine_data_icb(self):
        self.icb["OperatorID"] = self.icb["OperatorID"].astype(str)
        self.icb["operator_info"] = self.icb["OperatorID"].map(self.icb_operator_dict)
        self.icb["info_tuple"] = self.icb["PlateNumb"].map(self.car_info_dict)
        self.icb[["operator_name", "operator_name_en"]] = pd.DataFrame(self.icb["operator_info"].tolist(), index=self.icb.index, columns=["operator_name", "operator_name_en"])
        self.icb[["Brand", "Body", "Power_type", "Year"]] = pd.DataFrame(self.icb["info_tuple"].tolist(), index=self.icb.index, columns=["Brand", "Body", "Power_type", "Year"])
        self.icb["City"] = ""
        self.icb["Date"] = pd.to_datetime(self.icb["Date"], format="%Y-%m-%d")
        self.icb = self.icb.groupby([pd.Grouper(key="Date", freq="m"), "PlateNumb", "City", "operator_name", "operator_name_en", "Brand", "Body", "Power_type", "Year"])[["Distance", "Time"]].sum().reset_index()
        self.icb["Type"] = "Intercity Bus"
        return self.icb
    
    def run(self):
        cb = self.combine_data_cb()
        icb = self.combine_data_icb()
        result = pd.concat([cb, icb]).reset_index(drop=True)
        result["CreatedDateTime"] = pd.to_datetime(self.today)
        
        return result
    
    

In [26]:
df = DataConcat().df

In [62]:
cnxn = pyodbc.connect(driver='SQL Server',server="10.226.0.53",database="master", uid="BI_User", password="BsSQLDB")
cur = cnxn.cursor()

In [55]:
cur.execute("truncate table SAP_DW.dbo.KC_Data")
cnxn.commit()

In [58]:
for index, row in df.iterrows():
    cur.execute('INSERT INTO SAP_DW.dbo.KC_Data(Date, PlateNumb, City, operator_name, operator_name_en, Brand, Body, Power_type, Year, Distance, Time, Type, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))


In [60]:
cnxn.commit()

In [59]:
pd.read_sql("select * from SAP_DW.dbo.KC_Data", cnxn)

,Date,PlateNumb,City,operator_name,operator_name_en,Brand,Body,Power_type,Year,Distance,Time,Type,CreatedDateTime
0,2023-01-31,001-FT,高雄市,東南客運,"Southeast Bus Co., Ltd.",TOYOTA,整車進口,柴油,2012,3135.0,290.0,City Bus,2023-05-24
1,2023-01-31,001-U3,臺北市,大都會客運,"Metropolitan Bus Co., Ltd.",KINGLONG,KD-馨盛,柴油,2013,3002.9,283.7,City Bus,2023-05-24
2,2023-01-31,001-U7,臺中市,統聯客運,"United Highway Bus Co., Ltd.",大鳥金龍,金龍汽車,柴油,2015,4717.0,344.7,City Bus,2023-05-24
3,2023-01-31,001-U8,桃園市,統聯客運,"United Highway Bus Co., Ltd.",TOYOTA,整車進口,柴油,2013,3234.1,255.2,City Bus,2023-05-24
4,2023-01-31,001-V2,澎湖縣,澎湖縣公共車船管理處,"City Bus and Ferry Management Office, Penghu C...",HINO,馨盛,柴油,2020,2198.8,137.4,City Bus,2023-05-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70337,2023-05-31,KKB-8736,,屏東客運,"Pingtung Bus Co., Ltd.",HINO,馨盛,柴油,2015,2823.0,221.1,Intercity Bus,2023-05-24
70338,2023-05-31,KKB-8750,,屏東客運,"Pingtung Bus Co., Ltd.",HINO,協億,柴油,2016,2518.1,217.1,Intercity Bus,2023-05-24
70339,2023-05-31,KKB-8761,,屏東客運,"Pingtung Bus Co., Ltd.",HINO,,柴油,2015,0.0,0.0,Intercity Bus,2023-05-24
70340,2023-05-31,KKB-8766,,屏東客運,"Pingtung Bus Co., Ltd.",TOYOTA,整車進口,柴油,2022,2591.7,138.5,Intercity Bus,2023-05-24


In [61]:
cnxn.close()

In [44]:
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\TDX下載\客運業態數據.xlsx", index=False)